# Setup & File Load

In [ ]:
# import required packages
import os
import pandas as pd
import numpy
import mysql.connector
import transform


# set max number of columns & rows to display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

In [ ]:
# run this cell to load data from a csv file
df = pd.read_csv('./data/route_15A.csv', sep=";", na_values=['\\N'])

In [ ]:
# perform a check to see how many rows and columns are in the file
rows = df.shape[0]
cols = df.shape[1]
print()
print("Before any data cleaning, the CSV file contains", rows, "rows and", cols, "columns.")
print()

In [ ]:
df.head(5)

# Initial Checks on the Data

- Duplicate rows and columns
- Null/empty features
- Assign features as categorical or continuous
- Constant features

## Check for Duplicate Rows & Columns

In [ ]:
# Check for duplicate rows
print()
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])
# Check for duplicate columns
print('Duplicate columns:',df.columns.size - df.columns.unique().size)

There are no duplicate rows or columns so nothing needs to be dropped here.

## Check for Null/Empty Features

In [ ]:
# Perform a check for null/empty columns
df.describe().T

Features with count of zero can be dropped as they contain no information.

In [ ]:
# drop null columns
df = df.drop(columns=['tenderlot', 'suppressed_trip', 'justificationid_trip', 'passengers', 'passengersin', 'passengersout', 'distance_leavetimes', 'note_leavetimes', 'note_vehicle'])

## Assign Features as Continuous or Categorical

First check the data types of all rows after the file import.

In [ ]:
# print data types of all rows
df.dtypes

Assign categorical and continous features, and update the type of all categorical features to 'category'.

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['datasource', 'dayofservice', 'tripid', 'lineid', 'routeid', 'direction', 'basin', \
                         'lastupdate_trip', 'note_trip', 'progrnumber', 'stoppointid', \
                          'suppressed_leavetimes', 'lastupdate_leavetimes']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## Check for Constant Categorical Features

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

In [ ]:
# drop constant features
df = df.drop(columns=['datasource', 'basin'])

## Check for Constant Continuous Features

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no constant continuous features so nothing needs to be dropped.

# Further Analysis of Features

- Features that don't provide much information
- Features that we won't be able to provide information on to the model

In [ ]:
# drop features we won't use
df = df.drop(columns=['lastupdate_trip', 'note_trip', 'suppressed_leavetimes', 'justificationid_leavetimes', \
                      'lastupdate_leavetimes','vehicleid', 'distance_vehicle', 'minutes_vehicle'])

In [ ]:
df.head(5)

In [ ]:
df.dtypes

# Initial Checks for Missing Data

## Categorical Features

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction', 'progrnumber', 'stoppointid']].columns

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

There is a full count for all categorical features.

## Continuous Features

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are some rows missing data for **actualtime_arr_trip** and **actualtime_dep_trip**. This will be reviewed if these features are used in the future, currently they are not carried across when data is transformed.

# Transform the Data

In [ ]:
df_transformed = transform.transform_data(df)

In [ ]:
df_transformed

# Check for Missing Data

First re-assign the transformed data as continuous or categorical.

In [ ]:
df_transformed.dtypes

In [ ]:
# Select columns containing categorical data
categorical_columns = df_transformed[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction',  \
                         'progrnumber_first', 'stoppointid_first', \
                          'progrnumber_next', 'stoppointid_next']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_transformed[column] = df_transformed[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_transformed.select_dtypes(['float64', 'int64']).columns

Then check for missing data.

In [ ]:
# Print details for the categorical columns
df_transformed[categorical_columns].describe().T

In [ ]:
# Print details for the continuous columns
df_transformed[continuous_columns].describe().T

There are some rows with missing data, because the amount of missing rows is quite low, and because imputation would be difficult, these rows will be dropped.

## Drop Rows with Missing Data

In [ ]:
df_transformed = df_transformed[pd.notnull(df_transformed['stoppointid_first'])]

In [ ]:
df_transformed = df_transformed[pd.notnull(df_transformed['stoppointid_next'])]

In [ ]:
# Print details for the categorical columns
df_transformed[categorical_columns].describe().T

In [ ]:
# Print details for the continuous columns
df_transformed[continuous_columns].describe().T

# Import Weather Data

In [ ]:
# run this cell to load data directly from the database

# open connection
connection = mysql.connector.connect(host=os.environ['DBHOST'], user=os.environ['DBUSER'], \
        password=os.environ['DBPASS'], db='db_raw_data')

# SQL query 
sql = "SELECT * FROM weather_data \
WHERE record_date BETWEEN CAST('2018-01-01' AS DATE) AND CAST('2019-01-01' AS DATE);"

# load into dataframe
df_weather = pd.read_sql(sql, connection)

In [ ]:
# Check for duplicate rows
print()
print('Duplicate rows:', df_weather.duplicated()[df_weather.duplicated() == True].shape[0])
# Check for duplicate columns
print('Duplicate columns:',df_weather.columns.size - df_weather.columns.unique().size)

There are no duplicate rows or columns so nothing needs to be dropped here.

## Assign Features as Continuous or Categorical

First check the data types of all rows after the file import.

In [ ]:
df_weather.dtypes

Assign categorical and continuous features.

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'irain', 'itemp', 'iwb']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_weather.select_dtypes(['float64', 'int64']).columns

## Check for Missing Data, Constant Columns, etc.

In [ ]:
# Print details for the categorical columns
df_weather[categorical_columns].describe().T

**itemp** and **iwb** are constant columns so can be dropped.

In [ ]:
# Print details for the categorical columns
df_weather[continuous_columns].describe().T

Investigate rows with missing data for rain:

In [ ]:
# select all rows where irain is not 0
df_weather.loc[df_weather['irain'] != 0]

There are only two rows where irain is not zero, these rows correspond to missing values for rain.

In [ ]:
# select other rows around the missing values
df_weather[6220:6240]

Given that there is no rain for the rest of the day, and given the high (for Ireland) temperature on the day, I think it's safe to replace the missing rain values with 0.

I will then drop the feature **irain** as it provides no useful information.

### Replace missing data with 0

In [ ]:
# replace rain with 0 where irain is not 0
df_weather['rain'].loc[df_weather['irain'] != 0] = 0

In [ ]:
# check that values are updated
df_weather.loc[df_weather['irain'] != 0]

### Drop features

In [ ]:
# drop features we won't use
df_weather = df_weather.drop(columns=['irain', 'itemp', 'iwb'])

In [ ]:
df_weather

# Combine Bus and Weather Data

In [ ]:
from datetime import datetime

## Split date out for Weather Data

Define functions to split out month, day and hour.

In [ ]:
def timestamp_to_month(timestamp):
    timestamp = pd.to_datetime(timestamp)
    return timestamp.month

In [ ]:
def timestamp_to_day(timestamp):
    timestamp = pd.to_datetime(timestamp)
    return timestamp.day

In [ ]:
def timestamp_to_hour(timestamp):
    timestamp = pd.to_datetime(timestamp)
    return timestamp.hour

Add new columns using lambda functions.

In [ ]:
df_weather['month'] = df_weather.apply (lambda row: timestamp_to_month(row['record_date']), axis=1)

In [ ]:
df_weather['day'] = df_weather.apply (lambda row: timestamp_to_day(row['record_date']), axis=1)

In [ ]:
df_weather['hour'] = df_weather.apply (lambda row: timestamp_to_hour(row['record_date']), axis=1)

Update new columns to be categorical.

In [ ]:
# Select columns containing categorical data
categorical_columns = df_weather[['record_date', 'month', 'day', 'hour']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
df_weather.dtypes

## Split date out for Bus Data

Define functions to split out month, day and hour.

In [ ]:
df_transformed

In [ ]:
def timestamp_to_hour_bus(seconds):
    if seconds <= 86400:
        hour = seconds // 3600
    else:
        hour = (seconds - 86400) // 3600
    return hour % 24

In [ ]:
def timestamp_to_day_bus(timestamp, seconds):
    timestamp = pd.to_datetime(timestamp)
    if seconds < 86400:
        return timestamp.day
    else:
        if (timestamp.month in [1,3,5,7,8,10,12] and timestamp.day == 31) \
        or (timestamp.month in [4,6,9,11] and timestamp.day == 30) \
        or (timestamp.month == 2 and timestamp.day == 28):  # 2018 is not a leap year!
            return 1
        else:
            return timestamp.day + 1

In [ ]:
def timestamp_to_month_bus(timestamp, seconds):
    timestamp = pd.to_datetime(timestamp)

    if seconds >= 86400 and ((timestamp.month in [1,3,5,7,8,10,12] and timestamp.day == 31) \
    or (timestamp.month in [4,6,9,11] and timestamp.day == 30) \
    or (timestamp.month == 2 and timestamp.day == 28)):  # 2018 is not a leap year!
        return timestamp.month + 1
    else:
        return timestamp.month

Add new columns using lambda functions.

In [ ]:
df_transformed['month'] = df_transformed.apply (lambda row: timestamp_to_month_bus(row['dayofservice'], \
                                                                                   row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df_transformed['day'] = df_transformed.apply (lambda row: timestamp_to_day_bus(row['dayofservice'], \
                                                                               row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df_transformed['hour'] = df_transformed.apply (lambda row: timestamp_to_hour_bus(row['actualtime_arr_stop_first']), axis=1)

In [ ]:
df_transformed

## Merge the Dataframes

In [ ]:
df_merged = pd.merge(df_transformed, df_weather,  how='left', left_on=['month','day', 'hour'],\
                     right_on = ['month','day', 'hour'])

In [ ]:
# check that there are no rows missing weather data
df_merged[df_merged.rain.isnull()]

In [ ]:
df_merged

# Data Quality Plan - Bus Data (Before Transformation)

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| tenderlot       | All rows are null | Drop feature |
| suppressed_trip | All rows are null | Drop feature |
| justificationid_trip | All rows are null | Drop feature |
| passengers | All rows are null | Drop feature | 
| passengersin | All rows are null | Drop feature |
| passengersout | All rows are null | Drop feature |
| distance_leavetimes | All rows are null | Drop feature |
| note_leavetimes | All rows are null | Drop feature |
| note_vehicle | All rows are null | Drop feature |
| datasource | Constant feature | Drop feature |
| lineid | Constant feature | This is constant because we just have data for one route loaded. At some point we may process more than one route together so will keep feature for now. May not be needed to train the model. |
| basin | Constant feature | Drop feature |
| lastupdate_trip | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| note_trip | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| suppressed_leavetimes | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| justifcationid_leavetimes | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| lastupdate_leavetimes | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| vehicleid | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| distance_vehicle | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| minutes_vehicle | Cannot be used to train model as we won't be able to provide this information | Drop feature |
| actualtime_arr_trip | Missing values < 1% | Ignore for now as this feature is not brought across when data is transformed. |
| actualtime_dep_trip | Missing values < 3% | Ignore for now as this feature is not brought across when data is transformed. |

# Data Quality Plan - Bus Data (After Transformation)

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| stoppointid_first | Missing values ~ 1% | Drop affected rows |
| actualtime_arr_stop_first | Missing values ~ 1%| Drop affected rows |
| stoppointid_next | Missing values ~ 1% | Drop affected rows |
| actualtime_arr_stop_next | Missing values ~ 1%| Drop affected rows |

# Data Quality Plan - Weather Data

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| itemp | Constant feature | Drop feature |
| iwb | Constant feature | Drop feature |
| rain | Missing data - 2 rows | Imputation - replace with 0 after looking at data for other timestamps on the same day |
| irain | Seems to be a missing data indicator | Drop feature as only two rows have missing data, and imputation is performed for these rows. |

# Tests for Transforming the Data

In [ ]:
df_test1 = df.loc[5:100]
df_test1 = df_test1.reset_index(drop=True)
df_test1

In [ ]:
df_transformed1 = transform.transform_data(df_test1)
df_transformed1

In [ ]:
pieces = [df[:35], df[42:100]]
df_test2 = pd.concat(pieces)
df_test2 = df_test2.reset_index(drop=True)
df_test2

In [ ]:
df_transformed2 = transform.transform_data(df_test2)
df_transformed2

In [ ]:
pieces = [df[:5], df[10:100]]
df_test3 = pd.concat(pieces)
df_test3 = df_test3.reset_index(drop=True)
df_test3

In [ ]:
df_transformed3 = transform.transform_data(df_test3)
df_transformed3

In [ ]:
pieces = [df[:5], df[8:10], df[14:50]]
df_test4 = pd.concat(pieces)
df_test4 = df_test4.reset_index(drop=True)
df_test4

In [ ]:
df_transformed4 = transform.transform_data(df_test4)
df_transformed4